In [ ]:
import numpy as np
from hist import Hist
import hist
import matplotlib.pyplot as plt
import pickle

In [ ]:
edges = (5000, 185000)
entries_min = 100
entries_max = 1000

plates = range(1, 58)

bins = 360
scale = 180/bins  # size in mm

In [ ]:
with open(f"hists_{bins}.pkl", "rb") as f:
    h = pickle.load(f)

In [ ]:
hentries = Hist.new.Regular(100, entries_min, entries_max, name=rf"basetracks / ${scale**2} \mathrm{{mm}}^{{2}}$").Integer(1, 58, name="plate").Double()

In [ ]:
for plate in plates:
    h_p = h[:,:,plate - 1]
    plt.figure()
    h_p.plot()
    plt.savefig(f"plots/plate_{plate}.png")
    plt.close()
    hentries.fill(np.ravel(h_p), plate)

In [ ]:
plt.figure(figsize=(5,10))
hentries[:, plates[0]-1:plates[-1]-1].plot()
plt.savefig("plots/plate_preview.png")
plt.savefig("plots/plate_preview.pdf")

In [ ]:
for plate in plates:
    plt.figure()
    hentries[:, plate - 1].plot()
    plt.title(f"Plate {plate}")
    plt.savefig(f"plots/plate_{plate}_1d.png")
    plt.close()

In [ ]:
from scipy.stats import poisson

In [ ]:
for plate in [17,]:
    hentries_p = hentries[:, plate - 1]
    hentries_p.plot()
    h_poisson = Hist.new.Regular(100, entries_min, entries_max, name=rf"basetracks / ${scale**2} \mathrm{{mm}}^{{2}}$").Double()
    h_p = h[:,:,plate - 1]
    median = np.median(h_p.values())
    h_poisson.fill(poisson(median).rvs(size=int(hentries_p.integrate(0))))
    h_poisson.plot()
    plt.title("Comparison of seen distribution with poisson")
    plt.savefig(f"plots/poisson_{plate}.png")
    plt.savefig(f"plots/poisson_{plate}.pdf")

In [ ]:
h_mask = Hist.new.Regular(bins, edges[0], edges[1], name="x").Regular(bins, edges[0], edges[1], name="y").Integer(1, 58, name="plate").Double()

In [ ]:
for plate in plates:
    if plate == 23:
        continue
    hentries_p = hentries[:, plate - 1]
    h_p = h[:,:,plate - 1]
    median = np.median(h_p.values())
    cutoff = median + 3 * np.sqrt(median)
    for i in range(bins):
        for j in range(bins):
            h_mask[i,j,plate - 1] = (h_p[i,j] - cutoff) if h_p[i,j] > cutoff else 0

In [ ]:
for plate in plates:
    h_p = h_mask[:,:,plate - 1]
    plt.figure()
    h_p.plot()
    plt.savefig(f"plots/plate_{plate}_masked.png")
    plt.close()

In [ ]:
rebin_scale = 1
h_mask[hist.rebin(rebin_scale),hist.rebin(rebin_scale),:].project("x", "y").plot()
plt.xlabel(r"x / µm")
plt.ylabel(r"y / µm")
plt.title(rf"Basetracks exceeding threshold (integrated over plates) / ${(scale * rebin_scale)**2}\;\mathrm{{mm}}^2$")
plt.savefig("plots/projection_all_above_threshold_without_23.png")
plt.savefig("plots/projection_all_above_threshold_without_23.pdf")

In [ ]:
rebin_scale = 4
count = 10
h_mask_projected = h_mask[hist.rebin(rebin_scale),hist.rebin(rebin_scale),sum]
with open("candidates.csv", "a") as file:
    for place in np.argpartition(-h_mask_projected.values(), count, axis=None)[:count]:
        plt.figure(figsize=(15,5))
        indices = np.unravel_index(place, np.shape(h_mask_projected))
        print(indices)
        position = [5000 + indices[0] * rebin_scale * scale * 1000, 5000 + indices[1] * rebin_scale * scale * 1000]
        h_mask[hist.rebin(rebin_scale),hist.rebin(rebin_scale),:][*indices,:].plot()
        bin_x_low = position[0]
        bin_x_high = position[0] + 1000 * rebin_scale * scale
        bin_y_low = position[1]
        bin_y_high = position[1] + 1000 * rebin_scale * scale
        plt.title(rf"Shower candidate {place} bin{indices} ($x/\mu\mathrm{{m}} \in[{bin_x_low},{bin_x_high}), y/\mu\mathrm{{m}} \in [{bin_y_low},{bin_y_high})$ )")
        plt.ylabel(rf"Basetracks exceeding threshold / ${(scale * rebin_scale)**2}\;\mathrm{{mm}}^2$")
        plotname = f"candidate_{place}_{indices}_rebin_scale_{rebin_scale}_scale_{scale}"
        plt.savefig(f"plots/{plotname}.png")
        plt.savefig(f"plots/{plotname}.pdf")
        plt.close()
        file.write(f"{place}, {scale}, {rebin_scale}, {bin_x_low}, {bin_x_high}, {bin_y_low}, {bin_y_high}, {plotname}\n")